<a href="https://colab.research.google.com/github/atarasaki/DSWP-editados/blob/main/2020-11-23-LabDataChurn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## notebook para o Kaggle - construção em aula - 23/11/2020

In [1]:
## importa pycaret
!pip install pycaret

     |████████████████████████████████| 256kB 9.3MB/s 
     |████████████████████████████████| 66.3MB 60kB/s 
     |████████████████████████████████| 1.8MB 46.3MB/s 
     |████████████████████████████████| 102kB 9.9MB/s 
     |████████████████████████████████| 6.8MB 43.6MB/s 
     |████████████████████████████████| 13.9MB 275kB/s 
     |████████████████████████████████| 1.6MB 48.7MB/s 
     |████████████████████████████████| 148.9MB 78kB/s 
     |████████████████████████████████| 276kB 52.1MB/s 
     |████████████████████████████████| 266kB 52.8MB/s 
     |████████████████████████████████| 174kB 54.1MB/s 
     |████████████████████████████████| 2.1MB 47.0MB/s 
     |████████████████████████████████| 81kB 8.9MB/s 
     |████████████████████████████████| 153kB 48.5MB/s 
     |████████████████████████████████| 163kB 50.9MB/s 
     |████████████████████████████████| 1.1MB 43.8MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 337kB 49.5MB/s 

In [10]:
!pip install shap

     |████████████████████████████████| 327kB 11.6MB/s 
  Created wheel for shap: filename=shap-0.37.0-cp36-cp36m-linux_x86_64.whl size=463915 sha256=09396432bed8cd2a4232fd2775fe9695f8917e37df64433328a07a7efc52581f
  Stored in directory: /root/.cache/pip/wheels/df/ad/b0/aa7815ec68850d66551ef618095eccb962c8f6022f1d3dd989
Successfully built shap


In [11]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

import seaborn as sns
sns.set_theme(style='ticks', palette='pastel')

from pycaret import regression, classification
import shap

from collections import Counter

import tensorflow as tf
from tensorflow import keras

In [12]:
from pycaret.utils import enable_colab
enable_colab()

Colab mode enabled.


## Configurações de ambiente

In [15]:
pd_od = pd.options.display

In [17]:
pd_od.max_columns= 50
pd_od.max_rows = 50
pd_od.precision = 3

In [16]:
# alternativa
# pd.set_option('display.precision',3)
#pd.set_option('display.max_columns',50)
# pd.set_option('display.max_rows',50)

## Facets - DataViz

In [18]:
from IPython.core.display import display, HTML
import base64

In [21]:
!pip install facets-overview==1.0.0

In [28]:
from facets_overview.feature_statistics_generator import FeatureStatisticsGenerator

In [24]:
# treino
url_T = 'https://raw.githubusercontent.com/atarasaki/DSWP-editados/main/Datasets/Labdata%20Churn%20Challenge%202020/train.csv'

# validação/teste
url_V = 'https://raw.githubusercontent.com/atarasaki/DSWP-editados/main/Datasets/Labdata%20Churn%20Challenge%202020/test.csv'

In [25]:
df_T = pd.read_csv( url_T )

df_V = pd.read_csv( url_V)

In [26]:
df_T.head()

,id,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,4030,Female,0,No,No,56.0,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,NaN,45.05,2560.1,0
1,6731,Male,0,Yes,Yes,NaN,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,332.65,0
2,6479,Female,0,Yes,No,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.7,0
3,6861,Female,0,No,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,1
4,3266,Male,0,Yes,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.8,1


In [27]:
df_T.dtypes

id                    int64
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure              float64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                 int64
dtype: object

In [29]:
# Conversão de tipos : TotalCharges deveria ser float e está como object

df_T['TotalCharges2'] = pd.to_numeric( df_T['TotalCharges'], errors='coerce' )
df_T.dtypes

id                    int64
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure              float64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                 int64
TotalCharges2       float64
dtype: object

In [30]:
# aparentemente, conversão bem sucedida
df_T[['TotalCharges','TotalCharges2']].head()

,TotalCharges,TotalCharges2
0,2560.1,2560.10
1,332.65,332.65
2,6640.7,6640.70
3,3545.35,3545.35
4,1974.8,1974.80


In [31]:
# índice 161 havia sido reportado na conversão pd.to_numeric() sem a opção errors='coerce'
df_T.loc[161,['TotalCharges','TotalCharges2']]

TotalCharges        
TotalCharges2    NaN
Name: 161, dtype: object

In [35]:
# missing values nas colunas 'TotalCharges' e 'TotalCharges2'

df_T[['TotalCharges','TotalCharges2']].isnull().sum()

TotalCharges     0
TotalCharges2    8
dtype: int64